In [19]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from pylab import plot,show
from numpy import vstack,array
from numpy.random import rand
import numpy as np
from scipy.cluster.vq import kmeans,vq
import pandas_datareader as dr
from math import sqrt
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
import random

In [20]:
snp500url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
snp1000url = "https://en.wikipedia.org/wiki/List_of_S%26P_1000_companies"

#scraping wikipedia to fetch S&P 500 stock list
snp500_data_table = pd.read_html(snp500url)
snp1000_data_table = pd.read_html(snp1000url)

In [21]:
snp_500_tickers = snp500_data_table[0][1:]['Symbol'].tolist()
snp_1000_tickers = snp1000_data_table[3]['Ticker Symbol'].tolist()

print("Number of S&P500 Companies: " + str(len(snp_500_tickers)))
print("Number of S&P1000 Companies: " + str(len(snp_1000_tickers)))

KeyError: 'Ticker Symbol'

In [ ]:
large_cap_tickers = random.sample(snp_500_tickers, 50)
mid_and_small_cap_tickers = random.sample(snp_1000_tickers, 50)

In [ ]:
large_cap = []
for ticker in tqdm(large_cap_tickers):
    try:
        prices = dr.DataReader(ticker,'yahoo','01/01/2017')['Close']
        prices = pd.DataFrame(prices)
        prices.columns = [ticker]
        large_cap.append(prices)
    except:
        pass
    large_df = pd.concat(large_cap,axis=1)
large_df.sort_index(inplace=True)

In [ ]:
small_mid_cap = []
for ticker in tqdm(mid_and_small_cap_tickers):
    try:
        prices = dr.DataReader(ticker,'yahoo','01/01/2017')['Close']
        prices = pd.DataFrame(prices)
        prices.columns = [ticker]
        small_mid_cap.append(prices)
    except:
        pass
    small_mid_df = pd.concat(small_mid_cap,axis=1)
small_mid_df.sort_index(inplace=True)

In [ ]:
large_df.shape

In [ ]:
small_mid_df.shape

In [ ]:
small_mid_df.columns

In [ ]:
prices_df = pd.merge(large_df, small_mid_df, how='outer', left_index=True, right_index=True)

In [ ]:
prices_df.shape

In [ ]:
def get_returns(df):
    
    lis = prices_df.columns
    returns = []
    stdev = []
    cols = []
    srt = np.sqrt(252)
    
    for col in lis:
        ret_lis = df[col].pct_change()
        ret_lis = ret_lis.fillna(0)
        
        rets = ret_lis.mean()*252
        returns.append(rets)
        
        volt = ret_lis.std()*srt
        stdev.append(volt)
        
        cols.append(col)
    
    dfr = pd.DataFrame({'Company_Name': cols, 'Annual_Returns':returns, 'Volatility':stdev})
    dfr = dfr.set_index('Company_Name')
    
    return dfr

In [ ]:
df_returns = get_returns(prices_df)

In [ ]:
prices_df.columns.value_counts()

In [ ]:
large_df.columns

In [ ]:
## Initializing X and y

a = df_returns[['Annual_Returns','Volatility']]
X = np.array(a)

plt.rcParams['figure.figsize'] = (24, 13.5)
cost =[] 
for i in range(1, 11): 
    KM = KMeans(n_clusters = i, max_iter = 1000) 
    KM.fit(X) 
      
    # calculates squared error 
    # for the clustered points 
    cost.append(KM.inertia_)      

##plot the cost agaisnt K values

plt.plot(range(1, 11), cost, color ='g', linewidth ='3') 
plt.xlabel("Value of K") 
plt.ylabel("Sqaured Error (Cost)") 
plt.show() # clear the plot 

In [ ]:
## Clustering using K = 3

km = KMeans(
    n_clusters=3, init='random',
    n_init=10, max_iter=1000, 
    tol=1e-04, random_state=0
)
y_km = km.fit_predict(X)

In [ ]:
plt.scatter(
    X[y_km == 0, 0], X[y_km == 0, 1],
    s=50, c='lightgreen',
    marker='s', edgecolor='black',
    label='cluster 0')

plt.scatter(
    X[y_km == 1, 0], X[y_km == 1, 1],
    s=50, c='orange',
    marker='o', edgecolor='black',
    label='cluster 1')

plt.scatter(
    X[y_km == 2, 0], X[y_km == 2, 1],
    s=50, c='lightblue',
    marker='v', edgecolor='black',
    label='cluster 2')


# plot the centroids
plt.scatter(
    km.cluster_centers_[:, 0], km.cluster_centers_[:, 1],
    s=250, marker='*',
    c='red', edgecolor='black',
    label='centroids')

plt.xlabel('Returns')
plt.ylabel('Volatility')
plt.legend(scatterpoints=1)
plt.grid()
plt.show()

In [ ]:
df_returns['cluster'] = y_km

df_returns.cluster.value_counts()

In [ ]:
# Portfolio Cluster 0
company_ticker = df_returns.loc[df_returns.cluster==1].index.tolist()

In [ ]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2))

In [ ]:
LargeCapCluster0 = intersection(company_ticker, large_df.columns)
Mid_SmallCapCluster0 = intersection(company_ticker, small_mid_df.columns)

In [ ]:
Cluster0_df = large_df[LargeCapCluster0]
Cluster0_df = pd.merge(Cluster0_df, small_mid_df[Mid_SmallCapCluster0], left_index=True, right_index=True)

In [ ]:
Cluster0_df

In [ ]:
returns_daily = Cluster0_df.pct_change()

monthly_df = Cluster0_df.resample('BMS').first()
returns_monthly = monthly_df.pct_change().dropna()

In [ ]:
#find correlation matrix, i.e. the "distances" between each stock
#Load relevant packages
import datetime
from pandas_datareader import data as web
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm

corr = Cluster0_df.corr().abs()
size = 7
fig, ax = plt.subplots(figsize=(size, size))
ax.matshow(corr,cmap=cm.get_cmap('coolwarm'), vmin=0,vmax=1)
plt.xticks(range(len(corr.columns)), corr.columns, rotation='vertical', fontsize=8);
plt.yticks(range(len(corr.columns)), corr.columns, fontsize=8);

In [ ]:
lower

In [ ]:
# Select upper triangle of correlation matrix
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))
lower = corr.where(np.tril(np.ones(corr.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop_upper = [column for column in upper.columns if any(upper[column] > 0.95)]
to_drop_lower = [column for column in lower.columns if any(upper[column] > 0.95)]

In [ ]:
upper

In [ ]:
to_drop_upper

In [ ]:
to_drop_lower

In [ ]:
Cluster0_df.drop(Cluster0_df[to_drop], axis=1, inplace=True)

In [ ]:
Cluster0_df.shape

In [ ]:
# calculating daily and annual returns of the stocks
returns_daily = Cluster0_df.pct_change()
returns_annual = returns_daily.mean() * 250

# get daily and covariance of returns of the stock
cov_daily = returns_daily.cov()
cov_annual = cov_daily * 250

# empty lists to store returns, volatility and weights of imiginary portfolios
port_returns = []
port_volatility = []
sharpe_ratio = []
stock_weights = []

# set the number of combinations for imaginary portfolios
selected = Cluster0_df.columns
num_assets = len(selected)
num_portfolios = 50000

#set random seed for reproduction's sake
np.random.seed(101)

# populate the empty lists with each portfolios returns,risk and weights
for single_portfolio in range(num_portfolios):
    weights = np.random.random(num_assets)
    weights /= np.sum(weights)
    
    returns = np.dot(weights, returns_annual)
    volatility = np.sqrt(np.dot(weights.T, np.dot(cov_annual, weights)))
    
    sharpe = returns / volatility
    sharpe_ratio.append(sharpe)
    port_returns.append(returns)
    port_volatility.append(volatility)
    stock_weights.append(weights)

# a dictionary for Returns and Risk values of each portfolio
portfolio = {'Returns': port_returns,
             'Volatility': port_volatility,
             'Sharpe Ratio': sharpe_ratio}

# extend original dictionary to accomodate each ticker and weight in the portfolio
for counter,symbol in enumerate(selected):
    portfolio[symbol+' Weight'] = [Weight[counter] for Weight in stock_weights]

# make a nice dataframe of the extended dictionary
df = pd.DataFrame(portfolio)


# get better labels for desired arrangement of columns
column_order = ['Returns', 'Volatility', 'Sharpe Ratio'] + [stock+' Weight' for stock in selected]

# reorder dataframe columns
df = df[column_order]

# plot frontier, max sharpe & min Volatility values with a scatterplot
plt.style.use('seaborn-dark')
df.plot.scatter(x='Volatility', y='Returns', c='Sharpe Ratio',
                cmap='RdYlBu', edgecolors='black', figsize=(10, 8), grid=True)
plt.xlabel('Volatility')
plt.ylabel('Returns')
plt.title('Efficient Frontier')
plt.show()

In [ ]:
# finding min Volatility & max sharpe values in the dataframe (df)
min_volatility = df['Volatility'].min()
max_sharpe = df['Sharpe Ratio'].max()

# use the min, max values to locate and create the two special portfolios
sharpe_portfolio = df.loc[df['Sharpe Ratio'] == max_sharpe]
min_variance_port = df.loc[df['Volatility'] == min_volatility]

# plot frontier, max sharpe & min Volatility values with a scatterplot
plt.style.use('seaborn-dark')
df.plot.scatter(x='Volatility', y='Returns', c='Sharpe Ratio',
                cmap='RdYlBu', edgecolors='black', figsize=(10, 8), grid=True)

plt.scatter(x=sharpe_portfolio['Volatility'], y=sharpe_portfolio['Returns'], c='red', marker='*', s=200)
plt.scatter(x=min_variance_port['Volatility'], y=min_variance_port['Returns'], c='blue', marker='*', s=200 )
plt.xlabel('Volatility')
plt.ylabel('Returns')
plt.title('Efficient Frontier')
plt.show()

In [ ]:
sharpe_portfolio

In [ ]:
min_variance_port